# Data preprocessing script - CSTB

Authors: [Andreas Sørensen](https://www.linkedin.com/in/a-soerensen) and [Martin Röck](https://www.linkedin.com/in/martinroeck/) - Source: https://doi.org/10.5281/zenodo.5895051

In [1]:
# SETUP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import functools

In [2]:
# set data frame printing settings
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
print("Pre-processing CSTB data...")

# Import data

In [3]:
# IMPORT

#File name/path
filename = '0_data/0_data_input_raw/CSTB_data_input_raw.xlsx'

#Load all sheets
#all_sheets = pd.read_excel(filename, sheet_name=None)

#Load individual sheets
df_sheet_0 = pd.read_excel(filename, sheet_name='operations')
df_sheet_1 = pd.read_excel(filename, sheet_name='batiments')
df_sheet_2 = pd.read_excel(filename, sheet_name='contributeurs')
df_sheet_3 = pd.read_excel(filename, sheet_name='lots')
df_sheet_4 = pd.read_excel(filename, sheet_name='sous-lots')
df_sheet_5 = pd.read_excel(filename, sheet_name='Nouveaux indicateurs')

# Clean data

In [4]:
#Recreate dataframe with second header instead of first header (delete first header)
#This will also assign datatypes correctly
headers = df_sheet_0.iloc[0]
df_sheet_0  = pd.DataFrame(df_sheet_0.values[1:], columns=headers)
headers = df_sheet_1.iloc[0]
df_sheet_1  = pd.DataFrame(df_sheet_1.values[1:], columns=headers)
headers = df_sheet_2.iloc[0]
df_sheet_2  = pd.DataFrame(df_sheet_2.values[1:], columns=headers)
headers = df_sheet_3.iloc[0]
df_sheet_3  = pd.DataFrame(df_sheet_3.values[1:], columns=headers)
headers = df_sheet_4.iloc[0]
df_sheet_4  = pd.DataFrame(df_sheet_4.values[1:], columns=headers)
headers = df_sheet_5.iloc[0]
df_sheet_5  = pd.DataFrame(df_sheet_5.values[1:], columns=headers)

In [5]:
#Select relevant columns for each sheet
df_sheet_0 = df_sheet_0[['id_operation','date_etude_rsenv','phase']]
df_sheet_1 = df_sheet_1[['id_operation','id_batiment','usage_principal','type_travaux','nb_occupant','periode_etude_reference','sdp','nb_niv_ssol','nb_niv_surface','type_structure_principale','materiau_principal','type_toiture','niveau_energie','indicateur_1']]
df_sheet_2 = df_sheet_2[['id_operation','id_batiment','IND1_PCE','IND1_ENE','IND1_EAU','IND1_CHA']]
df_sheet_3 = df_sheet_3[['id_operation','id_batiment','IND1_LOT1','IND1_LOT2','IND1_LOT3','IND1_LOT4','IND1_LOT5','IND1_LOT6','IND1_LOT7','IND1_LOT8','IND1_LOT9','IND1_LOT10','IND1_LOT11','IND1_LOT12','IND1_LOT13','IND1_LOT14']]
#df_sheet_4 = df_sheet_4[['id_operation','id_batiment']]
df_sheet_5 = df_sheet_5[['id_operation','id_batiment','cef','co2']]

In [6]:
#Merge the sheets using the ID's
data_frames = [df_sheet_1,df_sheet_2,df_sheet_3,df_sheet_5]
#Merge sheet 1,2,3 and 4
df_CSTB = functools.reduce(lambda  left,right: pd.merge(left,right,on=['id_batiment','id_operation'],how='outer'), data_frames)
#Merge sheet 1-2-3-4 and 0
df_CSTB = pd.merge(df_CSTB,df_sheet_0,how='left',on='id_operation')

In [7]:
#Define valid cases to continue with based on provided list of valid operation ID's:
id_operation_valid = [188, 189, 192, 204, 211, 212, 214, 272, 413, 441, 464, 478, 483, 485, 489, 535, 537, 538, 539, 563, 564, 566, 567, 568, 569, 570, 573, 598, 600, 629, 652, 684, 699, 700, 701, 717, 718, 719, 733, 736, 740, 742, 743, 744, 747, 750, 751, 754, 755, 756, 762, 763, 764, 765, 766, 767, 771, 774, 775, 776, 778, 779, 780, 781, 784, 785, 786, 789, 790, 792, 794, 800, 816, 818, 819, 829, 830, 831, 832, 836, 843, 853, 857, 862, 867, 873, 877, 881, 883, 885, 887, 889, 890, 891, 893, 894, 896, 908, 914, 915, 921, 924, 925, 927, 931, 933, 941, 949, 950, 951, 952, 957, 958, 966, 971, 972, 974, 977, 978, 979, 980, 981, 983, 984, 985, 986, 987, 988, 989, 990, 991, 994, 996, 997, 1002, 1003, 1006, 1007, 1008, 1009, 1011, 1012, 1013, 1015, 1020, 1022, 1023, 1026, 1027, 1028, 1030, 1031, 1032, 1033, 1034, 1037, 1038, 1039, 1041, 1044, 1047, 1049, 1050, 1054, 1055, 1057, 1058, 1059, 1063, 1064, 1067, 1095, 1096, 1098, 1099, 1102, 1104, 1105, 1106, 1167, 1169, 1172, 1186, 1189, 1192, 1193, 1196, 1197, 1206, 1208, 1213, 1214, 1217, 1219, 1221, 1226, 1231, 1234, 1236, 1244, 1246, 1247, 1250, 1254, 1257, 1264, 1265, 1267, 1289, 1290, 1291, 1292, 1295, 1296, 1297, 1302, 1304, 1306, 1309, 1310, 1319, 1321, 1323, 1324, 1325, 1326, 1327, 1335, 1339, 1342, 1348, 1349, 1369, 1370, 1372, 1379, 1387, 1393, 1401, 1405, 1408, 1409, 1410, 1416, 1417, 1422, 1425, 1426, 1427, 1432, 1435, 1461, 1462, 1469, 1476, 1478, 1479, 1482, 1484, 1487, 1498, 1511, 1515, 1516, 1519, 1522, 1523, 1528, 1531, 1542, 1543, 1544, 1565, 1567, 1578, 1583, 1585, 1591, 1599, 1600, 1601, 1602, 1603, 1605, 1606, 1611, 1615, 1616, 1618, 1626, 1627, 1628, 1629, 1648, 1652, 1654, 1658, 1659, 1660, 1661, 1662, 1663, 1664, 1665, 1666, 1667, 1668, 1670, 1671, 1672, 1674, 1675, 1676, 1681, 1684, 1685, 1689, 1691, 1692, 1693, 1694, 1695, 1696, 1697, 1698, 1704, 1706, 1707, 1709, 1711, 1718, 1720, 1721, 1722, 1727, 1731, 1732, 1733, 1734, 1735, 1736, 1737, 1738, 1739, 1740, 1744, 1747, 1748, 1749, 1750, 1751, 1752, 1753, 1754, 1757, 1758, 1759, 1760, 1764, 1768, 1769, 1770, 1771, 1772, 1773, 1774, 1776, 1777, 1779, 1780, 1781, 1782, 1790, 1793, 1804, 1805, 1806, 1807, 1810, 1844, 1845, 1846, 1847,1849, 1851, 1852, 1853, 1854, 1857, 1858, 1860, 1862, 1863, 1864, 1868, 1875, 1877, 1878, 1885, 1890, 1892, 1896, 1898, 1899, 1902, 1907, 1910, 1911, 1914, 1920, 1925, 1926, 1927, 1929, 1930, 1932, 1945, 1947, 1954, 1968, 1969, 1970, 1971, 1972, 1979, 1980, 1982, 1983, 1984, 1986, 1987, 1991, 1993, 1996, 1997, 2008, 2010, 2011, 2013, 2014, 2015, 2017, 2019, 2026, 2027, 2032, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2044, 2046, 2048, 2051, 2052, 2053, 2054, 2055, 2058, 2059, 2065, 2067, 2072, 2073, 2076, 2077, 2079, 2081, 2082, 2083, 2089, 2095, 2098, 2115, 2116, 2122, 2123, 2124, 2127, 2130, 2132, 2138, 2142, 2144, 2145, 2146, 2149, 2152, 2153, 2154, 2156, 2158, 2160, 2167, 2168, 2170, 2173, 2174, 2178, 2180, 2182, 2187, 2189, 2190, 2191, 2192, 2193, 2197, 2204]
#NOTE: This could be imported as a file for less clutter in script.

In [8]:
#Create dataframe with the list
df_id_operation_valid = pd.DataFrame(id_operation_valid)

#Rename column to match id_operation in df_CSTB
dict_1 = {
  0: "id_operation"
}
df_id_operation_valid.rename(columns = dict_1, inplace = True)

In [9]:
#Filter the CSTB dataframe using df_id_operation_valid, so only valid entries remain:
df_CSTB = df_CSTB[df_CSTB.id_operation.isin(df_id_operation_valid.id_operation)]
#df_CSTB

In [10]:
#print('Number of cases after filtering away low quality cases as defined by CSTB:')
#df_CSTB.shape[0]

Number of cases after filtering away low quality cases as defined by CSTB:


712

In [11]:
#Drop all rows which have a missing value
#df_CSTB.dropna(how='any',inplace=True)

#Drop rows which have more than x missing values
x = 3
df_CSTB = df_CSTB[df_CSTB.isnull().sum(axis=1) < x]

#Data is cleaned

In [12]:
#Remove cases where "type_structure_principale" is 5 (invalid entry)
df_CSTB = df_CSTB[df_CSTB.type_structure_principale != 5]

In [13]:
df_CSTB = df_CSTB.apply(pd.to_numeric, errors='ignore') # convert all columns of DataFrame

In [14]:
#df_CSTB.dtypes

In [15]:
#print('Number of cases after removing cases with more than '+str(x)+' missing value(s):')
#df_CSTB.shape[0]

Number of cases after removing cases with more than 3 missing value(s):


487

In [16]:
df_CSTB

,id_operation,id_batiment,usage_principal,type_travaux,nb_occupant,periode_etude_reference,sdp,nb_niv_ssol,nb_niv_surface,type_structure_principale,materiau_principal,type_toiture,niveau_energie,indicateur_1,IND1_PCE,IND1_ENE,IND1_EAU,IND1_CHA,IND1_LOT1,IND1_LOT2,IND1_LOT3,IND1_LOT4,IND1_LOT5,IND1_LOT6,IND1_LOT7,IND1_LOT8,IND1_LOT9,IND1_LOT10,IND1_LOT11,IND1_LOT12,IND1_LOT13,IND1_LOT14,cef,co2,date_etude_rsenv,phase
47,188,279,Maison individuelle ou accolée,Bâtiments neufs,2.00,50,96.000000,0,0,Maçonnerie,Terre cuite,3 pans et plus,2,91796.94,65528.23,22985.79,3276.94,5.97,1626.11,4583.08,7439.64,4157.81,7388.89,5730.60,5223.15,12662.76,2781.97,4796.50,95.93,0.00,0.00,9041.79,15.8,1.1381,2017-05-09,Réalisation
48,189,280,Maison individuelle ou accolée,Bâtiments neufs,2.00,50,93.000000,0,0,Maçonnerie,Terre cuite,2 pans,2,90868.91,64156.09,23030.34,3676.51,5.97,2910.67,4154.59,6895.22,3163.44,8639.36,3911.36,5689.17,12281.28,2698.16,4652.00,93.04,0.00,0.00,9067.80,16.5,1.1851,2017-05-23,Réalisation
51,192,283,Maison individuelle ou accolée,Bâtiments neufs,2.00,50,104.000000,0,0,Maçonnerie,Terre cuite,2 pans,2,92338.69,63510.02,25625.58,3197.13,5.97,1192.12,4704.79,9615.48,3512.62,9491.55,4264.31,4411.40,13790.04,3029.63,5223.50,104.47,0.00,0.00,4170.10,NaN,NaN,2017-07-03,Réalisation
59,204,307,Logement collectif,Bâtiments neufs,30.00,50,1106.000000,1,4,Voiles porteurs,Béton,Terrasse,2,1066492.00,765970.60,235423.54,43511.75,21586.11,13982.69,127616.78,222899.56,22366.21,42212.01,42640.47,30641.36,84084.88,35404.16,50893.48,6638.28,48680.72,0.00,37910.00,19.7,1.3767,2016-12-20,Réalisation
60,204,308,Logement collectif,Bâtiments neufs,40.00,50,1518.000000,1,4,Voiles porteurs,Béton,Terrasse,2,1416588.15,1022347.30,310346.48,57638.74,26255.63,18557.61,173403.41,306222.16,31439.34,51525.74,53422.69,40173.95,115375.60,48579.20,69832.60,9108.60,66796.40,0.00,37910.00,18.8,1.3004,2016-12-20,Réalisation
62,211,315,Maison individuelle ou accolée,Bâtiments neufs,2.00,50,89.000000,0,0,Maçonnerie,Béton,2 pans,2,114783.14,59570.10,51730.25,2709.05,773.74,1965.55,4259.61,9988.55,903.39,5356.98,6241.91,3304.65,11811.36,2594.92,4474.00,89.48,536.88,0.00,8042.83,38.6,7.6024,2017-06-21,Réalisation
63,212,316,Maison individuelle ou accolée,Bâtiments neufs,2.00,50,101.000000,0,0,Maçonnerie,Béton,2 pans,2,118759.96,59667.11,55473.13,2845.99,773.74,1938.25,3635.08,9472.69,807.55,5648.33,4910.83,3182.66,13338.60,2930.45,5052.50,101.05,606.30,0.00,8042.83,40.1,8.0677,2017-06-21,Exploitation
64,214,318,Maison individuelle ou accolée,Bâtiments neufs,2.00,50,166.000000,0,2,Maçonnerie,Pierre,3 pans et plus,2,143649.46,108226.83,33183.49,2233.18,5.97,13332.54,13297.85,12801.89,3764.93,6240.33,10071.46,4941.29,21915.96,4814.87,8301.50,166.03,996.18,0.00,7582.00,NaN,NaN,2017-04-25,Exploitation
74,272,379,Maison individuelle ou accolée,Bâtiments neufs,2.00,50,96.000000,0,1,Maçonnerie,Béton,2 pans,2,80264.63,62447.64,15097.86,2713.16,5.97,9279.63,5245.10,8929.57,2266.66,3736.27,5158.68,6379.38,12706.32,2791.54,4813.00,96.26,0.00,0.00,1045.23,NaN,NaN,2017-10-23,Conception
90,413,527,Maison individuelle ou accolée,Bâtiments neufs,2.00,50,121.000000,0,1,Maçonnerie,Béton,2 pans,2,117099.85,78801.55,36321.60,1970.73,5.97,6722.52,7753.87,9301.20,5793.25,5673.72,8460.84,6999.46,15945.60,3503.20,6040.00,120.80,0.00,0.00,2487.10,NaN,NaN,2017-11-30,Programmation


# Rename directly transferable data

Description:
Rename column headers of data columns that are ready to be exported so they fit with Data Collection Template

In [17]:
#Translate "nb_occupant" to "bldg_users_total"

#Rename column header
df_CSTB.rename(columns = {"nb_occupant":"bldg_users_total"}, inplace = True)

In [18]:
#Translate "nb_niv_surface" to "bldg_floors_ag"

#Rename column header
df_CSTB.rename(columns = {"nb_niv_surface":"bldg_floors_ag"}, inplace = True)

In [19]:
#Translate "nb_niv_ssol" to "bldg_floors_bg"

#Rename column header
df_CSTB.rename(columns = {"nb_niv_ssol":"bldg_floors_bg"}, inplace = True)

In [20]:
#Translate "sdp" to "bldg_area_hfa"

#Rename column header
df_CSTB.rename(columns = {"sdp":"bldg_area_gfa"}, inplace = True)

In [21]:
#Translate "niveau_energie" to "bldg_energy_class_country"

#Rename column header
df_CSTB.rename(columns = {"niveau_energie":"bldg_energy_class_country"}, inplace = True)

In [22]:
#Translate "cef" to "inv_energy_consumption"

#Rename column header
df_CSTB.rename(columns = {"cef":"inv_energy_consumption"}, inplace = True)

In [23]:
#Translate "periode_etude_reference" to "lca_RSP"

#Rename column header
df_CSTB.rename(columns = {"periode_etude_reference":"lca_RSP"}, inplace = True)

In [24]:
#Translate "indicateur_1" to "GHG_sum_em"

#Rename column header
df_CSTB.rename(columns = {"indicateur_1":"GHG_sum_em"}, inplace = True)

#FOLLOWING HAS BEEN ADDED DUE TO INCORRECT CALC OTHERWISE

#Need to remove operational energy and water emisssions from total to get sum embodied:
df_CSTB["GHG_sum_em"] = df_CSTB["GHG_sum_em"]-df_CSTB["IND1_ENE"]-df_CSTB["IND1_EAU"]-df_CSTB["IND1_LOT1"]-df_CSTB["IND1_LOT14"]

Data columns have been renamed.
These data columns are now ready for export.

# Translate and regroup/rename data

Description:
Translate data entries that are written in French or need to be rephrased to fit the categories and formats of the Data Collection Template. Rename column headers so they fit with Data Collection Template.

In [25]:
#Translate "usage_principal" to "bldg_use_subtype"

try:
    #Display unique entries in "usage_principal" which need to be replaced
    #print(df_CSTB.usage_principal.unique())

    #Replace entries with English translation
    df_CSTB.usage_principal.replace('Maison individuelle ou accolée','Single family house',inplace=True)
    df_CSTB.usage_principal.replace('Logement collectif','Multi-family house',inplace=True)
    df_CSTB.usage_principal.replace('Bureaux','Office',inplace=True)
    df_CSTB.usage_principal.replace(['Enseignement primaire','Enseignement secondaire (partie jour)',"Etablissement d'accueil de la petite enfance (crèche, halte-garderie)",'Restauration scolaire - 1 repas/jour, 5j/7'],'School and Daycare',inplace=True)
    df_CSTB.usage_principal.replace("Bâtiment à usage d'habitation - Etablissement sanitaire avec hébergement",'Other',inplace=True)
    df_CSTB.usage_principal.replace("Hôpital (partie jour)",'Hospital and Health',inplace=True)
    
    #Rename column header
    df_CSTB.rename(columns = {"usage_principal":"bldg_use_subtype"}, inplace = True)

except:
    print('usage_principal already processed')

['Maison individuelle ou accolée' 'Logement collectif' 'Bureaux'
 "Etablissement d'accueil de la petite enfance (crèche, halte-garderie)"
 "Bâtiment à usage d'habitation - Etablissement sanitaire avec hébergement"
 'Enseignement secondaire (partie jour)' 'Hôpital (partie jour)'
 'Enseignement primaire' 'Restauration scolaire - 1 repas/jour, 5j/7']


In [26]:
#Translate "type_toiture" to "bldg_roof_type"

try:
    #Display unique entries in "type_toiture" which need to be replaced
    #print(df_CSTB.type_toiture.unique())
    
    #Replace entries with English translation
    df_CSTB.type_toiture.replace(['3 pans et plus',5],'Other',inplace=True)
    df_CSTB.type_toiture.replace('2 pans','Gable or saddle roof',inplace=True)
    df_CSTB.type_toiture.replace('Terrasse','Flat roof',inplace=True)
    df_CSTB.type_toiture.replace('Monopente','Single pitched roof',inplace=True)

    #Rename column header
    df_CSTB.rename(columns = {"type_toiture":"bldg_roof_type"}, inplace = True)

except:
    print('type_toiture already processed')

['3 pans et plus' '2 pans' 'Terrasse' 'Monopente']


In [27]:
#Translate "type_travaux" to "bldg_project_status"

try:
    #Display unique entries in "type_travaux" which need to be replaced
    #print(df_CSTB.type_travaux.unique())

    #Replace entries with English translation
    df_CSTB.type_travaux.replace('Bâtiments neufs','New Built',inplace=True)
    df_CSTB.type_travaux.replace('Extensions ou surélévations','Renovation',inplace=True)
    
    #Rename column header
    df_CSTB.rename(columns = {"type_travaux":"bldg_project_status"}, inplace = True)

except:
    print('type_travaux already processed')            

['Bâtiments neufs' 'Extensions ou surélévations']


In [28]:
#Translate "date_etude_rsenv" to "bldg_year_complete_interval"

try:
    df_CSTB['date_etude_rsenv'] = df_CSTB['date_etude_rsenv'].str[:4]
    
    #Display unique entries in "type_travaux" which need to be replaced
    #print(df_CSTB.date_etude_rsenv.unique())
    
    #Replace entries with English translation
    df_CSTB.date_etude_rsenv.replace(['2015','2016','2017','2018','2019'],'2015-2019',inplace=True)
    df_CSTB.date_etude_rsenv.replace('2020','2020-Today',inplace=True)

    #Rename column header
    df_CSTB.rename(columns = {"date_etude_rsenv":"bldg_year_complete_interval"}, inplace = True)

except:
    print('date_etude_rsenv already processed')   

['2017' '2016' '2018' '2015' '2019']


Data columns have been translated and renamed.
These data columns are ready for export.

# Transform and derive data through inference

In [29]:
#Transform and translate "type_structure_principale" and "materiau_principal" to "bldg_struct_type"

#Add new column as combination of "type_structure_principale" and "materiau_principal"
df_CSTB['bldg_struct_type'] = df_CSTB['type_structure_principale']+' '+df_CSTB['materiau_principal']

#Retrieve unique entries in "bldg_struct_type"
#print(df_CSTB.bldg_struct_type.unique())

#Replace entries with English translation
df_CSTB.bldg_struct_type.replace('Maçonnerie Terre cuite','massive brick',inplace=True)
df_CSTB.bldg_struct_type.replace(['Voiles porteurs Béton','Voiles porteurs Béton haute performance'],'massive concrete',inplace=True)
df_CSTB.bldg_struct_type.replace('Maçonnerie Béton','massive brick',inplace=True)
df_CSTB.bldg_struct_type.replace('Maçonnerie Autre, à préciser','massive brick',inplace=True)
df_CSTB.bldg_struct_type.replace('Ossature Bois massif','frame wood',inplace=True)
df_CSTB.bldg_struct_type.replace(['Poteaux/poutres Béton','Poteaux/poutres Béton haute performance'],'frame concrete',inplace=True)
df_CSTB.bldg_struct_type.replace('Maçonnerie Béton haute performance','massive brick',inplace=True)
df_CSTB.bldg_struct_type.replace('Poteaux/poutres Autre, à préciser','other',inplace=True)
df_CSTB.bldg_struct_type.replace('Poteaux/poutres Mixte: bois-béton','mix concrete wood',inplace=True)
df_CSTB.bldg_struct_type.replace('Ossature Bois massif reconstitué','frame wood',inplace=True)
df_CSTB.bldg_struct_type.replace('Ossature Mixte: bois-béton','frame concrete/wood',inplace=True)
df_CSTB.bldg_struct_type.replace('Poteaux/poutres Mixte: béton-acier','frame concrete/steel',inplace=True)
df_CSTB.bldg_struct_type.replace(['Poteaux/poutres Bois massif','Poteaux/poutres Bois massif reconstitué'],'frame wood',inplace=True)
df_CSTB.bldg_struct_type.replace(['Maçonnerie Bois massif reconstitué','Maçonnerie Mixte: bois-béton','Maçonnerie Bois massif'],'massive brick',inplace=True)
df_CSTB.bldg_struct_type.replace('Ossature Acier','frame steel',inplace=True)
df_CSTB.bldg_struct_type.replace('Voiles porteurs Béton cellulaire','massive concrete',inplace=True)
df_CSTB.bldg_struct_type.replace('Ossature Béton','frame concrete',inplace=True)
df_CSTB.bldg_struct_type.replace('Maçonnerie Pierre','massive brick',inplace=True)
df_CSTB.bldg_struct_type.replace('Voiles porteurs Terre cuite','massive brick',inplace=True)
df_CSTB.bldg_struct_type.replace('Maçonnerie Béton cellulaire','massive brick',inplace=True)
df_CSTB.bldg_struct_type.replace('Maçonnerie Bois massif','massive brick',inplace=True)
df_CSTB.bldg_struct_type.replace('Ossature Terre cuite','massive brick',inplace=True)
df_CSTB.bldg_struct_type.replace(['Voiles porteurs Bois massif reconstitué','Voiles porteurs Bois massif'],'massive wood',inplace=True)


['Maçonnerie Terre cuite' 'Voiles porteurs Béton' 'Maçonnerie Béton'
 'Maçonnerie Pierre' 'Maçonnerie Autre, à préciser' 'Ossature Bois massif'
 'Poteaux/poutres Béton' 'Maçonnerie Béton haute performance'
 'Ossature Béton' 'Ossature Bois massif reconstitué'
 'Ossature Mixte: bois-béton' 'Poteaux/poutres Autre, à préciser'
 'Poteaux/poutres Béton haute performance'
 'Maçonnerie Bois massif reconstitué' 'Ossature Acier'
 'Ossature Terre cuite' 'Voiles porteurs Béton cellulaire'
 'Voiles porteurs Terre cuite' 'Maçonnerie Béton cellulaire'
 'Voiles porteurs Bois massif reconstitué' 'Voiles porteurs Bois massif'
 'Maçonnerie Mixte: bois-béton' 'Maçonnerie Bois massif'
 'Poteaux/poutres Bois massif reconstitué'
 'Voiles porteurs Béton haute performance']


In [30]:
#Translate "materiau_principal" to "inv_mat_1_type" (Placed here due to dependency above)

try:
    #Display unique entries in "materiau_principal" which need to be replaced
    #print(df_CSTB.materiau_principal.unique())

    #Replace entries with English translation
    df_CSTB.materiau_principal.replace('Terre cuite','Ceramics (e.g., fired clay bricks)',inplace=True)
    df_CSTB.materiau_principal.replace('Béton','Concrete w/o reinforcement',inplace=True)
    df_CSTB.materiau_principal.replace('Pierre','Stone (granite, limestone, etc)',inplace=True)
    df_CSTB.materiau_principal.replace('Autre, à préciser','Other',inplace=True)
    df_CSTB.materiau_principal.replace('Bois massif','Timber, wood',inplace=True)
    df_CSTB.materiau_principal.replace('Béton haute performance','Concrete w/o reinforcement',inplace=True)
    df_CSTB.materiau_principal.replace('Mixte: bois-béton','Other',inplace=True)
    df_CSTB.materiau_principal.replace('Bois massif reconstitué','Timber, wood',inplace=True)
    df_CSTB.materiau_principal.replace('Mixte: béton-acier','Other',inplace=True)
    df_CSTB.materiau_principal.replace('Acier','Metals (iron, steel)',inplace=True)
    df_CSTB.materiau_principal.replace('Béton cellulaire','Other',inplace=True)  

    #Rename column header
    df_CSTB.rename(columns = {"materiau_principal":"inv_mat_1_type"}, inplace = True)

except:
    print('materiau_principal already processed')  

['Terre cuite' 'Béton' 'Pierre' 'Autre, à préciser' 'Bois massif'
 'Béton haute performance' 'Bois massif reconstitué' 'Mixte: bois-béton'
 'Acier' 'Béton cellulaire']


In [31]:
#Add infered and calculated columns

#Infer admin_project_code
df_CSTB['admin_project_code'] = range(1,1+len(df_CSTB))
df_CSTB['admin_project_code'] = 'CSTB'+df_CSTB['admin_project_code'].astype(str)

#Infer admin_project_contact
df_CSTB['admin_project_contact'] = 'CSTB'

#Infer bldg_use_type (Building type)
#Copy "bldg_use_subtype"
df_CSTB['bldg_use_type'] = df_CSTB['bldg_use_subtype']
#Replace entries with infered type
df_CSTB.bldg_use_type.replace(['Single family house','Multi-family house'],'Residential',inplace=True)
df_CSTB.bldg_use_type.replace(['Office','School and Daycare','Hospital and Health'],'Non-residential',inplace=True)

#Infer site_country (Country of construction)
df_CSTB['site_country'] = 'France'

#Infer building area definition
df_CSTB['bldg_area_definition'] = 'France'

#Infer building area interval
#df_CSTB['bldg_area_interval'] = df_CSTB['bldg_area_gfa']
df_CSTB.loc[df_CSTB['bldg_area_gfa'] <= 5000, 'bldg_area_interval'] = '0-5000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 5000) & (df_CSTB['bldg_area_gfa'] <= 10000), 'bldg_area_interval'] = '5001-10000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 10000) & (df_CSTB['bldg_area_gfa'] <= 15000), 'bldg_area_interval'] = '10001-15000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 15000) & (df_CSTB['bldg_area_gfa'] <= 20000), 'bldg_area_interval'] = '15001-20000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 20000) & (df_CSTB['bldg_area_gfa'] <= 25000), 'bldg_area_interval'] = '20001-25000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 25000) & (df_CSTB['bldg_area_gfa'] <= 30000), 'bldg_area_interval'] = '25001-30000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 30000) & (df_CSTB['bldg_area_gfa'] <= 35000), 'bldg_area_interval'] = '30001-35000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 35000) & (df_CSTB['bldg_area_gfa'] <= 40000), 'bldg_area_interval'] = '35001-40000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 40000) & (df_CSTB['bldg_area_gfa'] <= 45000), 'bldg_area_interval'] = '40001-45000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 45000) & (df_CSTB['bldg_area_gfa'] <= 50000), 'bldg_area_interval'] = '45001-50000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 50000) & (df_CSTB['bldg_area_gfa'] <= 55000), 'bldg_area_interval'] = '50001-55000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 55000) & (df_CSTB['bldg_area_gfa'] <= 60000), 'bldg_area_interval'] = '55001-60000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 60000) & (df_CSTB['bldg_area_gfa'] <= 65000), 'bldg_area_interval'] = '60001-65000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 65000) & (df_CSTB['bldg_area_gfa'] <= 70000), 'bldg_area_interval'] = '65001-70000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 70000) & (df_CSTB['bldg_area_gfa'] <= 75000), 'bldg_area_interval'] = '70001-75000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 75000) & (df_CSTB['bldg_area_gfa'] <= 80000), 'bldg_area_interval'] = '75001-80000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 80000) & (df_CSTB['bldg_area_gfa'] <= 85000), 'bldg_area_interval'] = '80001-85000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 85000) & (df_CSTB['bldg_area_gfa'] <= 90000), 'bldg_area_interval'] = '85001-90000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 90000) & (df_CSTB['bldg_area_gfa'] <= 95000), 'bldg_area_interval'] = '90001-95000'
df_CSTB.loc[(df_CSTB['bldg_area_gfa'] > 95000) & (df_CSTB['bldg_area_gfa'] <= 100000), 'bldg_area_interval'] = '95001-100000'
df_CSTB.loc[df_CSTB['bldg_area_gfa'] > 100000, 'bldg_area_interval'] = '>10000'

#Infer LCA software
df_CSTB['lca_software'] = 'Other'

#Infer LCA database
df_CSTB['lca_database'] = 'INIES'

#Infer scope LCS
df_CSTB['scope_LCS_A123'] = 'Yes'
df_CSTB['scope_LCS_A4'] = 'Yes'
df_CSTB['scope_LCS_A5'] = 'Yes'
df_CSTB['scope_LCS_B1'] = 'Yes'
df_CSTB['scope_LCS_B2'] = 'Yes'
df_CSTB['scope_LCS_B3'] = 'Yes'
df_CSTB['scope_LCS_B4'] = 'Yes'
df_CSTB['scope_LCS_B5'] = 'Yes'
df_CSTB['scope_LCS_B6'] = 'Yes'
df_CSTB['scope_LCS_B7'] = 'Yes'
df_CSTB['scope_LCS_B8'] = 'No'
df_CSTB['scope_LCS_C1'] = 'Yes'
df_CSTB['scope_LCS_C2'] = 'Yes'
df_CSTB['scope_LCS_C3'] = 'Yes'
df_CSTB['scope_LCS_C4'] = 'Yes'
df_CSTB['scope_LCS_D'] = 'Yes'
df_CSTB['scope_handling_D'] = 'separately considered'

#Infer GHG_sum_m2a (Sum)
df_CSTB['GHG_sum_em_m2a'] = df_CSTB['GHG_sum_em']/df_CSTB['bldg_area_gfa']/df_CSTB['lca_RSP']

#Infer GHG_sum_em_m2a (Sum Operational)
df_CSTB['GHG_sum_op_m2a'] = df_CSTB['co2']

# "Lots" decoding work

We only need the "lots" from indicator 1 (IND1). There are 14 "lots" to be decoded.

Lot1: Roadwork and diverse

Lot2: Foundations and infrastructure

Lot3: Structure and masonry

Lot4: Roof and cover

Lot5: Interior partitions, suspended ceilings, components and joineries

Lot6: Exterior surfaces (facades), components (doors and windows) and joineries

Lot7: Interior coatings (floors, walls and ceilings)

Lot8: HVAC equipment

Lot9: Sanitary installations

Lot10: Electrical equipment

Lot11: Special electrical equipment (systems, controls and communication)

Lot12: Interior transport equipment (elevators etc.)

Lot13: Local electricity production equipment

Lot14: Unknown (not represented in method description)

Lot1 and Lot14 are not represented in our data collection template and will not be included.
It should be noted that these are included in the total results, and thus actions to remove these for more representative data could be taken.

In [32]:
#Use the "lots" to derive the building parts included in each row

#1 Ground, Substructure = Lot2: Foundations and infrastructure
df_CSTB["scope_parts_1_ground"] = ["No" if (i == 0 or i == np.nan) else "Yes" for i in df_CSTB["IND1_LOT2"]]

#2 Structure primary elements, carcass = Lot3: Structure and masonry or Lot4: Roof and cover
df_CSTB['scope_parts_2_structure'] = ["No" if (i == 0 or i == np.nan) else "Yes" for i in df_CSTB["IND1_LOT3"]]

#3 Secondary elements, openings = Lot6: Exterior surfaces (facades), components (doors and windows) and joineries
df_CSTB['scope_parts_3_secondary'] = ["No" if (i == 0 or i == np.nan) else "Yes" for i in df_CSTB["IND1_LOT6"]]

#4 Finishes = Lot5: Interior partitions, suspended ceilings, components and joineries or Lot7: Interior coatings (floors, walls and ceilings)
df_CSTB['scope_parts_4_finishes'] = ["No" if (i == 0 or i == np.nan) else "Yes" for i in df_CSTB["IND1_LOT7"]]

#5 Services, mainly mechanical = Lot8: HVAC equipment
df_CSTB['scope_parts_5_mechanical'] = ["No" if (i == 0 or i == np.nan) else "Yes" for i in df_CSTB["IND1_LOT8"]]

#6 Services, mainly electrical = Lot10: Electrical equipment or Lot11: Special electrical equipment (systems, controls and communication)
df_CSTB['scope_parts_6_electrical'] = ["No" if (i == 0 or i == np.nan) else "Yes" for i in df_CSTB["IND1_LOT10"]]

#6+ Renewable Energy Systems = Lot13: Local electricity production equipment
df_CSTB['scope_parts_6+_renewables'] = ["No" if (i == 0 or i == np.nan) else "Yes" for i in df_CSTB["IND1_LOT13"]]

#7 Facilities = Lot9: Sanitary installations
df_CSTB['scope_parts_7_facilities'] = ["No" if (i == 0 or i == np.nan) else "Yes" for i in df_CSTB["IND1_LOT9"]]

#8 Fittings is not present
df_CSTB['scope_parts_8_fittings'] = "No"

In [33]:
df_CSTB["scope_parts_1_ground"]

47      Yes
48      Yes
51      Yes
59      Yes
60      Yes
62      Yes
63      Yes
64      Yes
74      Yes
90      Yes
102     Yes
108     Yes
122     Yes
140     Yes
142     Yes
143     Yes
144     Yes
154     Yes
155     Yes
157     Yes
158     Yes
159     Yes
160     Yes
161     Yes
163     Yes
171     Yes
173     Yes
184     Yes
202     Yes
208     Yes
209     Yes
210     Yes
211     Yes
213     Yes
214     Yes
215     Yes
216     Yes
217     Yes
218     Yes
219     Yes
220     Yes
221     Yes
224     Yes
225     Yes
226     Yes
227     Yes
228     Yes
232     Yes
233     Yes
234     Yes
235     Yes
236     Yes
237     Yes
239     Yes
241     Yes
242     Yes
243     Yes
245     Yes
246     Yes
247     Yes
248     Yes
249     Yes
250     Yes
251     Yes
259     Yes
261     Yes
262     Yes
268     Yes
275     Yes
276     Yes
278     Yes
279     Yes
280     Yes
282     Yes
283     Yes
284     Yes
285     Yes
286     Yes
287     Yes
288     Yes
289     Yes
293     Yes
320     Yes
321 

In [34]:
#Use the "lots" to calculate the impact of individual building parts

#1 Ground = Lot2: Foundations and infrastructure
df_CSTB["GHG_P1_sum_m2a"] = (df_CSTB["IND1_LOT2"])/df_CSTB['bldg_area_gfa']/df_CSTB['lca_RSP']

#2 Structure = Lot3: Structure and masonry + Lot4: Roof and cover
df_CSTB["GHG_P2_sum_m2a"] = (df_CSTB["IND1_LOT3"] + df_CSTB["IND1_LOT4"])/df_CSTB['bldg_area_gfa']/df_CSTB['lca_RSP']

#3-4 Envelope = Lot6: Exterior surfaces (facades), components (doors and windows) and joineries
df_CSTB["GHG_P34_sum_m2a"] = (df_CSTB["IND1_LOT6"])/df_CSTB['bldg_area_gfa']/df_CSTB['lca_RSP']

#4 Internal = Lot5: Interior partitions, suspended ceilings, components and joineries + Lot7: Interior coatings (floors, walls and ceilings)
df_CSTB["GHG_P4_sum_m2a"] = (df_CSTB["IND1_LOT5"] + df_CSTB["IND1_LOT7"])/df_CSTB['bldg_area_gfa']/df_CSTB['lca_RSP']

#5-6 Services = Lot8: HVAC equipment + Lot10: Electrical equipment + Lot11: Special electrical equipment (systems, controls and communication) + Lot13: Local electricity production equipment
df_CSTB["GHG_P56_sum_m2a"] = (df_CSTB["IND1_LOT8"] + df_CSTB["IND1_LOT10"] + df_CSTB["IND1_LOT11"] + df_CSTB["IND1_LOT13"])/df_CSTB['bldg_area_gfa']/df_CSTB['lca_RSP']

#7-8 Appliances = Lot9: Sanitary installations
df_CSTB["GHG_P78_sum_m2a"] = (df_CSTB["IND1_LOT9"])/df_CSTB['bldg_area_gfa']/df_CSTB['lca_RSP']

# Add empty data columns

In [35]:
#Add empty columns
#(i.e. add columns from Data Collection sheet that aren't represented in CSTB for completeness)

#Time data (maybe we could infer this using time of the conduction of the study? -> date_etude_rsenv)
df_CSTB['bldg_year_permit'] = 'n/a'
df_CSTB['bldg_year_complete'] = 'n/a'
#df_CSTB['bldg_year_complete_interval'] = 'No data' #Now infered bases on date_etude_rsenv

#Project data status (Could be available in dataset in "phase" and "phase_exacte" but there are two and can't discerne between them)
df_CSTB['bldg_QTO_type'] = 'n/a'

#Gross floor area and area interval(This could be available in the dataset, need to check)
df_CSTB['bldg_area_hfa'] = 'n/a'
#df_CSTB['bldg_area_interval'] = 'n/a'

#Energy performance class (could be infered using the available energy class)
df_CSTB['bldg_energy_class_general'] = 'n/a'

#Sustainability certification (Niveau Carbone? Is it a certification scheme?)
df_CSTB['bldg_certification'] = 'n/a'

#Total mass of the building
df_CSTB['inv_mat_mass_total'] = 'n/a'

#Top 5 most used materials
#df_CSTB['inv_mat_1_type'] = 'n/a' (has been added)
df_CSTB['inv_mat_1_mass'] = 'n/a'
df_CSTB['inv_mat_2_type'] = 'n/a'
df_CSTB['inv_mat_2_mass'] = 'n/a'
df_CSTB['inv_mat_3_type'] = 'n/a'
df_CSTB['inv_mat_3_mass'] = 'n/a'
df_CSTB['inv_mat_4_type'] = 'n/a'
df_CSTB['inv_mat_4_mass'] = 'n/a'
df_CSTB['inv_mat_5_type'] = 'n/a'
df_CSTB['inv_mat_5_mass'] = 'n/a'

#LCA software (could be acquired from enquiry with data providers or infered from "RSEnv version")
#df_CSTB['lca_software'] = 'n/a'

#LCA database (could be acquired from enquiry with data providers or infered from "RSEnv version")
#df_CSTB['lca_database'] = 'n/a'

#Future decarbonisation scenarios considered
df_CSTB['lca_scenarios_decarbonisation'] = 'No data'

#Building parts included (have been added)
"""
df_CSTB['scope_parts_1_ground'] = 'No data'
df_CSTB['scope_parts_2_structure'] = 'No data'
df_CSTB['scope_parts_3_secondary'] = 'No data'
df_CSTB['scope_parts_4_finishes'] = 'No data'
df_CSTB['scope_parts_5_mechanical'] = 'No data'
df_CSTB['scope_parts_6_electrical'] = 'No data'
df_CSTB['scope_parts_6+_renewables'] = 'No data'
df_CSTB['scope_parts_7_facilities'] = 'No data'
df_CSTB['scope_parts_8_fittings'] = 'No data'
df_CSTB['scope_parts_9_external'] = 'No data'
"""

#Life cycle stages considered in the study (have been added)
"""
df_CSTB['scope_LCS_A123'] = 'No data'
df_CSTB['scope_LCS_A4'] = 'No data'
df_CSTB['scope_LCS_A5'] = 'No data'
df_CSTB['scope_LCS_B1'] = 'No data'
df_CSTB['scope_LCS_B2'] = 'No data'
df_CSTB['scope_LCS_B3'] = 'No data'
df_CSTB['scope_LCS_B4'] = 'No data'
df_CSTB['scope_LCS_B5'] = 'No data'
df_CSTB['scope_LCS_B6'] = 'No data'
df_CSTB['scope_LCS_B7'] = 'No data'
df_CSTB['scope_LCS_B8'] = 'No data'
df_CSTB['scope_LCS_C1'] = 'No data'
df_CSTB['scope_LCS_C2'] = 'No data'
df_CSTB['scope_LCS_C3'] = 'No data'
df_CSTB['scope_LCS_C4'] = 'No data'
df_CSTB['scope_LCS_D'] = 'No data'
df_CSTB['scope_handling_D'] = 'No data'
"""

#Results totals (Some totals have been added)
#df_CSTB['GHG_sum_em'] = 'n/a'
df_CSTB['GHG_sum_op'] = 'n/a'
#df_CSTB['GHG_sum_em_m2a'] = 'n/a'
#df_CSTB['GHG_sum_op_m2a'] = 'n/a'

#Results individual modules [kgCO2]
df_CSTB['GHG_A1'] = 'n/a'
df_CSTB['GHG_A2'] = 'n/a'
df_CSTB['GHG_A3'] = 'n/a'
df_CSTB['GHG_A4'] = 'n/a'
df_CSTB['GHG_A5'] = 'n/a'
df_CSTB['GHG_B1'] = 'n/a'
df_CSTB['GHG_B2'] = 'n/a'
df_CSTB['GHG_B3'] = 'n/a'
df_CSTB['GHG_B4'] = 'n/a'
df_CSTB['GHG_B5'] = 'n/a'
df_CSTB['GHG_B6'] = 'n/a'
df_CSTB['GHG_B7'] = 'n/a'
df_CSTB['GHG_C1'] = 'n/a'
df_CSTB['GHG_C2'] = 'n/a'
df_CSTB['GHG_C3'] = 'n/a'
df_CSTB['GHG_C4'] = 'n/a'
df_CSTB['GHG_D'] = 'n/a'

#Results individual modules [kgCO2/m2/y]
df_CSTB['GHG_A1_m2a'] = 'n/a'
df_CSTB['GHG_A2_m2a'] = 'n/a'
df_CSTB['GHG_A3_m2a'] = 'n/a'
df_CSTB['GHG_A4_m2a'] = 'n/a'
df_CSTB['GHG_A5_m2a'] = 'n/a'
df_CSTB['GHG_B1_m2a'] = 'n/a'
df_CSTB['GHG_B2_m2a'] = 'n/a'
df_CSTB['GHG_B3_m2a'] = 'n/a'
df_CSTB['GHG_B4_m2a'] = 'n/a'
df_CSTB['GHG_B5_m2a'] = 'n/a'
df_CSTB['GHG_B6_m2a'] = 'n/a'
df_CSTB['GHG_B7_m2a'] = 'n/a'
df_CSTB['GHG_C1_m2a'] = 'n/a'
df_CSTB['GHG_C2_m2a'] = 'n/a'
df_CSTB['GHG_C3_m2a'] = 'n/a'
df_CSTB['GHG_C4_m2a'] = 'n/a'
df_CSTB['GHG_D_m2a'] = 'n/a'

#Results aggregated modules [kgCO2]
df_CSTB["GHG_A123"] = 'n/a'
df_CSTB["GHG_A45"] = 'n/a'
df_CSTB["GHG_A12345"] = 'n/a'
df_CSTB["GHG_B1234"] = 'n/a'
df_CSTB["GHG_B12345"] = 'n/a'
df_CSTB["GHG_B67"] = 'n/a'
df_CSTB["GHG_B1234567"] = 'n/a'
df_CSTB["GHG_C12"] = 'n/a'
df_CSTB["GHG_C34"] = 'n/a'
df_CSTB["GHG_C1234"] = 'n/a'
df_CSTB["GHG_C34_D"] = 'n/a'
df_CSTB["GHG_C1234_D"] = 'n/a'

#Results aggregated modules [kgCO2/m2/y]
df_CSTB["GHG_A123_m2a"] = 'n/a'
df_CSTB["GHG_A45_m2a"] = 'n/a'
df_CSTB["GHG_A12345_m2a"] = 'n/a'
df_CSTB["GHG_B1234_m2a"] = 'n/a'
df_CSTB["GHG_B12345_m2a"] = 'n/a'
df_CSTB["GHG_B67_m2a"] = 'n/a'
df_CSTB["GHG_B1234567_m2a"] = 'n/a'
df_CSTB["GHG_C12_m2a"] = 'n/a'
df_CSTB["GHG_C34_m2a"] = 'n/a'
df_CSTB["GHG_C1234_m2a"] = 'n/a'
df_CSTB["GHG_C34_D_m2a"] = 'n/a'
df_CSTB["GHG_C1234_D_m2a"] = 'n/a'

#Results building parts
#1 Ground
#df_CSTB["GHG_P1_sum_m2a"] = 'n/a'
df_CSTB["GHG_P1_A123_m2a"] = 'n/a'
df_CSTB["GHG_P1_A45_m2a"] = 'n/a'
df_CSTB["GHG_P1_B1234_m2a"] = 'n/a'
df_CSTB["GHG_P1_B5_m2a"] = 'n/a'
df_CSTB["GHG_P1_C12_m2a"] = 'n/a'
df_CSTB["GHG_P1_C34_m2a"] = 'n/a'
df_CSTB["GHG_P1_D_m2a"] = 'n/a'

#2 Structure
#df_CSTB["GHG_P2_sum_m2a"] = 'n/a'
df_CSTB["GHG_P2_A123_m2a"] = 'n/a'
df_CSTB["GHG_P2_A45_m2a"] = 'n/a'
df_CSTB["GHG_P2_B1234_m2a"] = 'n/a'
df_CSTB["GHG_P2_B5_m2a"] = 'n/a'
df_CSTB["GHG_P2_C12_m2a"] = 'n/a'
df_CSTB["GHG_P2_C34_m2a"] = 'n/a'
df_CSTB["GHG_P2_D_m2a"] = 'n/a'

#3-4 Envelope
#df_CSTB["GHG_P34_sum_m2a"] = 'n/a'
df_CSTB["GHG_P34_A123_m2a"] = 'n/a'
df_CSTB["GHG_P34_A45_m2a"] = 'n/a'
df_CSTB["GHG_P34_B1234_m2a"] = 'n/a'
df_CSTB["GHG_P34_B5_m2a"] = 'n/a'
df_CSTB["GHG_P34_C12_m2a"] = 'n/a'
df_CSTB["GHG_P34_C34_m2a"] = 'n/a'
df_CSTB["GHG_P34_D_m2a"] = 'n/a'

#4 Internal
#df_CSTB["GHG_P4_sum_m2a"] = 'n/a'
df_CSTB["GHG_P4_A123_m2a"] = 'n/a'
df_CSTB["GHG_P4_A45_m2a"] = 'n/a'
df_CSTB["GHG_P4_B1234_m2a"] = 'n/a'
df_CSTB["GHG_P4_B5_m2a"] = 'n/a'
df_CSTB["GHG_P4_C12_m2a"] = 'n/a'
df_CSTB["GHG_P4_C34_m2a"] = 'n/a'
df_CSTB["GHG_P4_D_m2a"] = 'n/a'

#5-6 Services
#df_CSTB["GHG_P56_sum_m2a"] = 'n/a'
df_CSTB["GHG_P56_A123_m2a"] = 'n/a'
df_CSTB["GHG_P56_A45_m2a"] = 'n/a'
df_CSTB["GHG_P56_B1234_m2a"] = 'n/a'
df_CSTB["GHG_P56_B5_m2a"] = 'n/a'
df_CSTB["GHG_P56_C12_m2a"] = 'n/a'
df_CSTB["GHG_P56_C34_m2a"] = 'n/a'
df_CSTB["GHG_P56_D_m2a"] = 'n/a'

#7-8 Apppliances
#df_CSTB["GHG_P78_sum_m2a"] = 'n/a'
df_CSTB["GHG_P78_A123_m2a"] = 'n/a'
df_CSTB["GHG_P78_A45_m2a"] = 'n/a'
df_CSTB["GHG_P78_B1234_m2a"] = 'n/a'
df_CSTB["GHG_P78_B5_m2a"] = 'n/a'
df_CSTB["GHG_P78_C12_m2a"] = 'n/a'
df_CSTB["GHG_P78_C34_m2a"] = 'n/a'
df_CSTB["GHG_P78_D_m2a"] = 'n/a'

# Rearrange columns (drop columns)

In [36]:
#Reorder columns in CSTB dataframe using double brackets (columns that aren't called are dropped):
df_CSTB_processed = df_CSTB[[
    'admin_project_code',
    'admin_project_contact',
    'bldg_use_type',
    'bldg_use_subtype',
    'bldg_project_status',
    'site_country',
    'bldg_year_permit',
    'bldg_year_complete',
    'bldg_year_complete_interval',
    'bldg_QTO_type',
    'bldg_area_definition',
    'bldg_area_gfa','bldg_area_hfa',
    'bldg_area_interval',
    'bldg_users_total',
    'bldg_floors_ag',
    'bldg_floors_bg',
    'bldg_struct_type',
    'bldg_roof_type',
    'bldg_energy_class_general',
    'bldg_energy_class_country',
    'bldg_certification',
    
    'inv_energy_consumption',
    'inv_mat_mass_total',
    'inv_mat_1_type',
    'inv_mat_1_mass',
    'inv_mat_2_type',
    'inv_mat_2_mass',
    'inv_mat_3_type',
    'inv_mat_3_mass',
    'inv_mat_4_type',
    'inv_mat_4_mass',
    'inv_mat_5_type',
    'inv_mat_5_mass',
    
    'lca_RSP',
    'lca_software',
    'lca_database',
    'lca_scenarios_decarbonisation',
    'scope_parts_1_ground',
    'scope_parts_2_structure',
    'scope_parts_3_secondary',
    'scope_parts_4_finishes',
    'scope_parts_5_mechanical',
    'scope_parts_6_electrical',
    'scope_parts_6+_renewables',
    'scope_parts_7_facilities',
    'scope_parts_8_fittings',
    'scope_LCS_A123',
    'scope_LCS_A4',
    'scope_LCS_A5',
    'scope_LCS_B1',
    'scope_LCS_B2',
    'scope_LCS_B3',
    'scope_LCS_B4',
    'scope_LCS_B5',
    'scope_LCS_B6',
    'scope_LCS_B7',
    'scope_LCS_B8',
    'scope_LCS_C1',
    'scope_LCS_C2',
    'scope_LCS_C3',
    'scope_LCS_C4',
    'scope_LCS_D',
    'scope_handling_D',
    
    'GHG_sum_em',
    'GHG_sum_op',
    'GHG_sum_em_m2a',
    'GHG_sum_op_m2a',
    
    'GHG_A1',
    'GHG_A2',
    'GHG_A3',
    'GHG_A4',
    'GHG_A5',
    'GHG_B1',
    'GHG_B2',
    'GHG_B3',
    'GHG_B4',
    'GHG_B5',
    'GHG_B6',
    'GHG_B7',
    'GHG_C1',
    'GHG_C2',
    'GHG_C3',
    'GHG_C4',
    'GHG_D',
    
    'GHG_A1_m2a',
    'GHG_A2_m2a',
    'GHG_A3_m2a',
    'GHG_A4_m2a',
    'GHG_A5_m2a',
    'GHG_B1_m2a',
    'GHG_B2_m2a',
    'GHG_B3_m2a',
    'GHG_B4_m2a',
    'GHG_B5_m2a',
    'GHG_B6_m2a',
    'GHG_B7_m2a',
    'GHG_C1_m2a',
    'GHG_C2_m2a',
    'GHG_C3_m2a',
    'GHG_C4_m2a',
    'GHG_D_m2a',
    
    'GHG_A123',
    'GHG_A45',
    'GHG_A12345',
    'GHG_B1234',
    'GHG_B12345',
    'GHG_B67',
    'GHG_B1234567',
    'GHG_C12',
    'GHG_C34',
    'GHG_C1234',
    'GHG_C34_D',
    'GHG_C1234_D',  
    
    'GHG_A123_m2a',
    'GHG_A45_m2a',
    'GHG_A12345_m2a',
    'GHG_B1234_m2a',
    'GHG_B12345_m2a',
    'GHG_B67_m2a',
    'GHG_B1234567_m2a',
    'GHG_C12_m2a',
    'GHG_C34_m2a',
    'GHG_C1234_m2a',
    'GHG_C34_D_m2a',
    'GHG_C1234_D_m2a',
    
    'GHG_P1_sum_m2a',
    'GHG_P1_A123_m2a',
    'GHG_P1_A45_m2a',
    'GHG_P1_B1234_m2a',
    'GHG_P1_B5_m2a',
    'GHG_P1_C12_m2a',
    'GHG_P1_C34_m2a',
    'GHG_P1_D_m2a',
    'GHG_P2_sum_m2a',
    'GHG_P2_A123_m2a',
    'GHG_P2_A45_m2a',
    'GHG_P2_B1234_m2a',
    'GHG_P2_B5_m2a',
    'GHG_P2_C12_m2a',
    'GHG_P2_C34_m2a',
    'GHG_P2_D_m2a',
    'GHG_P34_sum_m2a',
    'GHG_P34_A123_m2a',
    'GHG_P34_A45_m2a',
    'GHG_P34_B1234_m2a',
    'GHG_P34_B5_m2a',
    'GHG_P34_C12_m2a',
    'GHG_P34_C34_m2a',
    'GHG_P34_D_m2a',
    'GHG_P4_sum_m2a',
    'GHG_P4_A123_m2a',
    'GHG_P4_A45_m2a',
    'GHG_P4_B1234_m2a',
    'GHG_P4_B5_m2a',
    'GHG_P4_C12_m2a',
    'GHG_P4_C34_m2a',
    'GHG_P4_D_m2a',
    'GHG_P56_sum_m2a',
    'GHG_P56_A123_m2a',
    'GHG_P56_A45_m2a',
    'GHG_P56_B1234_m2a',
    'GHG_P56_B5_m2a',
    'GHG_P56_C12_m2a',
    'GHG_P56_C34_m2a',
    'GHG_P56_D_m2a',
    'GHG_P78_sum_m2a',
    'GHG_P78_A123_m2a',
    'GHG_P78_A45_m2a',
    'GHG_P78_B1234_m2a',
    'GHG_P78_B5_m2a',
    'GHG_P78_C12_m2a',
    'GHG_P78_C34_m2a',
    'GHG_P78_D_m2a']]

# Export

In [38]:
#Create CSV
filename = '0_data/1_data_pre_processed/CSTB_processed.csv'
df_CSTB_processed.to_csv(filename, index=False)

In [ ]:
print("CSTB data pre-processed.")